# Adding Agents with RAGS

## Introduction

In this lab, you'll harness the power of Retrieval-Augmented Generation (RAG) and combine it with your custom Google_map tool from Lab2. Your challenge is to integrate these two components: use the Google_map tool to answer location-based queries, and leverage RAG (from Lab3) to provide context-rich answers from your company policy document. By merging tool use and RAG, you'll build an advanced agent capable of both retrieving document knowledge and interacting with external tools for dynamic, real-world tasks.

<table><tr>
<td><img src="../../images/robo1.png" alt="Robo1 - RAG Expert" width="120" /></td>
<td style="vertical-align:top; padding-left:20px;">
<b>Robo1 says:</b><br>
<i>Can you do WFH today? Ask me.. I know the answer</i>
</td>
</tr></table>


## Challenge: Create a RAG from Company Policy Document

Your task is to build a Retrieval-Augmented Generation (RAG) system using the company policy document. 

# Import Libraries and Setup
Let's import the necessary libraries and set up our environment for RAG-powered resume Q&A.

In [5]:
import googlemaps
from langchain.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
import logging
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter

### Load OpenAPI Key from .env File

To securely access the OpenAPI key, we will load it from a `.env` file using environment variable management tools. This ensures sensitive credentials are not exposed in the notebook.

In [2]:
# Load environment variables from .env file
load_dotenv()

# Get OpenAI API key from environment
openai_api_key = os.getenv("OPENAI_API_KEY")

### Create a Retreiver
- load your company policy PDF 
- split it into smaller chunks (embeddings)
- create a vector store from embeddings
- create a retriever

In [3]:
pdf_path = "company_policy.pdf"
loader = PyPDFLoader(pdf_path)
docs = loader.load()

# Split resume into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(docs)

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(split_docs, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})


### Create a tool using Google Map API

In [6]:
google_map_key= os.getenv("GOOGLE_MAPS_API_KEY")
gmaps = googlemaps.Client(key=google_map_key)

@tool
def directions_tool(origin: str, destination: str, mode: str = "driving") -> str:
    """Get directions between two places using Google Maps (modes: driving, walking, bicycling, transit)."""
    try:
       directions = gmaps.directions(origin, destination, mode)
       instructions = []
       for step in directions[0]["legs"][0]["steps"]:
            instructions.append(step["html_instructions"])
       return "\n".join(instructions)
    except Exception as e:
        return f"❌ Error fetching directions: {str(e)}"

### Intialize LLM with OpenAPI Key
- Intialize LLM
- Create RetreievalQA

In [7]:
llm = ChatOpenAI(model="gpt-4", temperature=0, api_key=openai_api_key)
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

### Create Appropriate Prompt Template

In [ ]:
prompt_template = """
You are an HR assistant for TechNova Solutions Pvt. Ltd. Answer company policy questions using the company policy document (RAG).
If the answer is not found in the document, reply: "I don't know based on the company document."
If the question involves locations, directions, or travel, use the Google Map directions tool to provide helpful information.
Always be clear, concise, and professional.
Question: {input}
{agent_scratchpad}
Answer as if you are the company HR.:
"""

### Create Agent
- Create Agent
- Create Agent Executor

In [9]:
agent = create_tool_calling_agent(llm, [directions_tool], PromptTemplate.from_template(prompt_template))
executor = AgentExecutor(agent=agent, tools=[directions_tool])

### Chat with Your Company Policy Agent
Ask questions about your company policy.

In [10]:
import asyncio

async def async_chat_loop(executor):
    print("Company Policy RAG Chatbot (Async Mode). Type your question (or 'exit' to quit):")
    while True:
        question = input("You: ")
        if question.lower() == "exit":
            print("Goodbye!")
            break
        print(f"[DEBUG] Agent received question: {question}")
        response = await asyncio.to_thread(executor.invoke, {"input": question})
        print("Bot:", response.get("output", response))

# To run the chat loop, uncomment the line below:
await async_chat_loop(executor)

Company Policy RAG Chatbot (Async Mode). Type your question (or 'exit' to quit):
[DEBUG] Agent received question: Can I do WFH today?
Bot: I don't know based on the company document.
[DEBUG] Agent received question: What is WFH Policy?
Bot: As a candidate, I don't have access to the specific company document to provide the WFH (Work From Home) policy. However, generally, a WFH policy would outline the company's rules and expectations for employees who work from home or another location outside of the office. It might include details about hours of work, communication methods, and productivity expectations. I would recommend asking the HR department or your supervisor for the specific WFH policy for this company.
Goodbye!


---

<table><tr>
<td><img src="../../images/robo1.png" alt="Robo1 - RAG Expert" width="120" /></td>
<td style="vertical-align:top; padding-left:20px;">
<b>Robo1 says:</b><br>
<i>"Congratulations! You just built a RAG-powered chatbot."</i><br>
<i>With RAG, your AI can find answers in documents faster than ever. 📄🤖</i>
</td>
</tr></table>

*Thanks for completing the LangChain RAG Lab!*